# 10 Feature Visualization

## 1
Just examining the kernels isn't very satisfying, because they tend to be small and numerous, and particularly difficult to understand when they're not being applied to the input image: In deeper layers of the network, we may be able to look at the kernels, but without considering previous feature maps in the image, we don't know what that kernel will *activate* on.  Perhaps one kernel might like to look for locations where cat ears are in close proximity to one another: the output of the previous layer might contain a kernel that activates on cat ears, but unless we know which kernel that is already, if we look at our current kernel we'll just know that it activates on when *something* is in close proximity.  

However, all hope is not lost.  An alternative task that we might consider is to find the RGB image that causes a given kernel in a given layer to *activate as strongly as possible*.  **How might you approach solving this problem, i.e. finding this image?  What should be the objective function?  What are the things we get to adjust?**

## 2 VGG
To be honest, the CNNs that we've worked with so far aren't very interesting.  They've been trained on relatively small datasets, and have pretty few parameters.  They are, for all intents and purposes, toy examples, and their behaviour is pretty uninteresting.  Why don't we use bigger and more modern networks as teaching examples?  They take too long to train!  However, for real industrial scale applications, many interesting CNNs already exist and are available *pre-trained*, and we can query their behavior.  One notable example is the neural network VGG16, which has the following architecture:
<img src="vgg.png" />
This is a so-called *deep network* because it has many layers.  It is also interesting because it doesn't do much fancy stuff (take advanced ML or Computer Vision if you want to know about fancy stuff), and the features that it extracts end up being relatively comprehensible.  Note that VGG was trained on (a subset of) the [ImageNet](http://www.image-net.org/) dataset, which contains 10M labelled images.  VGG was trained on more than a million of these and can recognize 1k different classes.  It's very large: 138M parameters and was trained for three weeks on an array of four Titan XP GPUs.  

We have straightforward access to it via pytorch, specifically the torchvision package.  It might take you a bit of time to download the weights the first time.  

In [ ]:
import torchvision.models as tvm
model = tvm.vgg16(pretrained=True).eval()

Now that we have VGG, we'd like to create a mechanism for solving the optimization problem that we figured out above: For a given kernel, find the image that maximizes the mean of the feature map produced by that kernel.  The following class has that capability.  

In [ ]:
import torch
import numpy as np
from scipy.ndimage import gaussian_filter

from PIL import Image
from torchvision import transforms
from torch.autograd import Variable

class FeatureVisualizer(object):
    """Class for visualizing features in the pretrained pytorch VGG model"""
    def __init__(self,model):
        """Pass in the correct model"""
        self.model = model  
           
    def create_feature_maximizer(self,layer_index,kernel_index,n_steps=100,gaussian_blur=0.1,imsize=224):
        """Finds an image that maximizes the activations induced by a given kernel
        Arguments:
           layer_index: the index of the layer in question
           kernel_index: the index of the kernel in that layer
           n_steps: number of optimization steps to take
           gaussian_blur: how much to blur the resulting image to minimize high frequency noise
           imsize: how big an image to make
        Outputs: 
           img: the image that maximizes the given activation
        """
        
        # Instantiate a random image of the appropriate size and convert it to a variable
        img = torch.from_numpy(np.uint8(np.random.uniform(-125, 125, (3,imsize, imsize)))/255)
        img = img.to(torch.float32).detach().unsqueeze(0)
        img = img.clone().detach().requires_grad_(True)

        # Instantiate an optimizer, with the pixel values of the input image as the variable to be optimized
        optimizer = torch.optim.Adam([img], lr=1e-1,weight_decay=1e-6)

        # Optimize
        for i in range(n_steps):
            # Zero the gradient buffer
            optimizer.zero_grad()
            
            # Create a data structure to store intermediate network outputs
            outputs = []
            def hook_fn(module, input, output):
                outputs.append(output)
            
            # Create a function that will store a given layer's output
            hook = model.features[layer_index].register_forward_hook(hook_fn)
            
            # Run the model on the current image value
            output = model(img)
            
            # Define the loss using the layer output
            loss = -outputs[0][0,kernel_index].mean()
            
            # Print some stats
            print("Negative Mean activation for layer:",layer_index,", kernel index", kernel_index,": ",loss.item())
            
            # Compute the gradient
            loss.backward()
            
            # Update the pixel values
            optimizer.step()
            
            # Destroy the intermediate output function
            hook.remove()
            
        # Convert img to w x h x channel, convert to numpy, remove batch dimension
        img = np.moveaxis(img.detach().numpy().squeeze(),0,2)
        
        # Normalize to [0,1] and blur
        img = gaussian_filter(np.dstack(([(img[:,:,i] - img[:,:,i].min())/(img[:,:,i].max() - img[:,:,i].min()) for i in range(3)])),gaussian_blur)
        
        return img
    
    def get_mean_layer_activations(self,image_path,layer_index,imsize=224):
        """Computes the mean activations of a layer for a given image
        Arguments:
          image_path: the path to the image we want to run the network on
          layer_index: the layer that we want to compute the mean activations for
        Outputs:
          mean_layer_activations: the mean of each feature map in a layer after being evaluated on an image
        """
        # Load and normalize image to format expected by VGG network
        loader = transforms.Compose([transforms.Resize(imsize),transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
        
        # Define data structure to hold intermediate network output
        outputs= []
        def hook_fn(module, input, output):
            outputs.append(output)
        
        # Define a function to load the image
        def image_loader(image_name):
            """load image, returns cuda tensor"""
            image = Image.open(image_name)
            image = loader(image).float()
            image = Variable(image, requires_grad=True)
            image = image.unsqueeze(0)
            return image

        # Load the image
        img = image_loader(image_path)

        # Run the model and save intermediate output
        hook = self.model.features[layer_index].register_forward_hook(hook_fn)
        output = self.model(img)
        hook.remove()
        mean_layer_activations = np.mean(outputs[0].detach().cpu().numpy().squeeze(),axis=(1,2))
        return mean_layer_activations


Now, we need to determine what layer to query.  We can do this by looking at the model architecture:

In [ ]:
model.features

Let's see what type of image the first kernel in the first convolutional layer (after applying relu) is looking for 

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
import matplotlib.pyplot as plt

layer = 1
kernel = 0
fv = FeatureVisualizer(model)
img = fv.create_feature_maximizer(layer,kernel,imsize=128,n_steps=200)


## IC10A Examining features at different layers
Produce images like the ones above for several other kernels in this CNN layer.  What type of features does this layer seem to be focusing on.  Do the same for different layers.  What can you say about the complexity of the features being detected at deeper levels of the neural network?

## IC10B 
We can also examine what the network is focusing when extracting features *for a specific image*.  Download your own image of a common object (a cat, or a truck, or something simple), and use the *get_mean_layer_activations* method in the class given above to get the mean activations for a given layer as a function of the kernel index.  Identify which kernel for a given layer is most strongly activated by your particular image.  Then, generate the image that maximizes the activation for that layer and kernel.  What feature of your chosen image is the network looking at?

In [ ]:
#e.g.
activations = fv.get_mean_layer_activations('cat.jpg',1)
plt.plot(activations)